# Gluing and Merging

Within this notebook, we glue the individual files which each contain a single sweep (elevation level), into full volume scans, where each file represents a volume scan. 

We have downloaded the raw data from the [ARM Data portal](https://adc.arm.gov/discovery/#/results/datastream::gucxprecipradarS2.00), using the `gucxprecipradarS2.00` datastream.

In [1]:
import os
import pyart
from matplotlib import pyplot as plt
import numpy as np
import time
import datetime
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


## 1) Define Functions

In [2]:
def radar_glue(b_radar, radar_list):
    for rad in radar_list:
        b_radar = pyart.util.join_radar(b_radar, rad)
    
    return b_radar

In [3]:
def volume_from_list(vlist, base_dir):
    base_radar = pyart.io.read(base_dir+vlist[0])
    radars = [pyart.io.read(base_dir+sw) for sw in vlist[1::]]
    return radar_glue(base_radar, radars)

In [4]:
def granule(Dvolume):
    DATA_DIR = '/Users/jrobrien/ARM/data/CSU-XPrecipRadar/raw/tmp/'
    if len(Dvolume) == 8:
        base_rad = pyart.io.read(DATA_DIR+Dvolume[0])
        out_radar = volume_from_list(Dvolume, DATA_DIR)
        print(out_radar.time['units'][14:])
        ff = time.strptime(out_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
        dt = datetime.datetime.fromtimestamp(time.mktime(ff)) + datetime.timedelta(seconds = int(out_radar.time['data'][0]))
        strform = dt.strftime(DATA_DIR + 'xprecipradar_guc_volume_%Y%m%d-%H%M%S.b1.nc')
        print(strform)
        #FIX for join issue.. to be fixed in Py-ART
        out_radar.sweep_mode['data']=np.tile(base_rad.sweep_mode['data'], N_TILTS)
        pyart.io.write_cfradial(strform, out_radar)

## 2) Define Processing Variables

In [5]:
# Define location of the raw data - NOTE: Must be untarred!
DATA_DIR = '/Users/jrobrien/ARM/data/CSU-XPrecipRadar/raw/tmp/'
# Define the suffix of the base scan
BASE_SCAN_PPI = '1_PPI.nc'
# Define the desired suffix of the volume file
PPI_PATTERN = 'PPI.nc'
# Define the number of elevation levels
N_TILTS = 8

## 3) Create Volume Scans

In [6]:
# sort the input files
all_files = os.listdir(DATA_DIR)
all_files.sort()

In [7]:
# Iterate over the files within the directory.
# Determine which are base scans and which are ppi scans
# NOTE: There are RHI scans within the tar file not used.
base_scans = []
volumes = []
ppis = []
in_volume = False
for file in all_files:
    if PPI_PATTERN in file:
        ppis.append(file)
    if BASE_SCAN_PPI in file:
        base_scans.append(file)

In [8]:
# Determine the scan volumes
volumes = []
for base in base_scans:
    base_scan_index = np.where(np.array(ppis) == base)[0][0]
    volume = ppis[base_scan_index: base_scan_index + N_TILTS]
    volumes.append(volume)

In [9]:
# Start up a Dask Cluster for Processing the Granule function
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()

cluster.scale(16)  # Sets the number of workers to 10
cluster.adapt(minimum=8, maximum=16)
client = Client(cluster)
client

/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/tornado/platform/asyncio.py:193: DeprecationWarning: There is no current event loop
  old_loop = asyncio.get_event_loop()
2022-09-15 15:19:47,835 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/7q/czgy7g_j3fb7jw3256f37wrw0000gq/T/dask-worker-space/worker-nj13a322', purging
2022-09-15 15:19:47,835 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/7q/czgy7g_j3fb7jw3256f37wrw0000gq/T/dask-worker-space/worker-jaqygonm', purging
2022-09-15 15:19:47,835 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/7q/czgy7g_j3fb7jw3256f37wrw0000gq/T/dask-worker-space/worker-7u5ppj24', purging
2022-09-15 15:19:47,835 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/7q/czgy7g_j3fb7jw3256f37wrw0000gq/T/dask-worker-space/worker-6zzwnxhv', purging
2022-09-15 15:19:47,836 - distributed.disku

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 32,Total memory: 102.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54791,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 102.40 GiB
Comm: tcp://127.0.0.1:54821,Total threads: 2
Dashboard: http://127.0.0.1:54822/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:54797,


In [10]:
# Use Dask distributed map utility to call the granule function
future = client.map(granule, volumes)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: Deprecation

In [11]:
my_data = client.gather(future)

/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: DeprecationWarning: the rpc synchronous context manager is deprecated
  with self.rpc(self.nanny) as r:
/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: DeprecationWarning: the rpc synchronous context manager is deprecated
  with self.rpc(self.nanny) as r:
/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: DeprecationWarning: the rpc synchronous context manager is deprecated
  with self.rpc(self.nanny) as r:
/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: DeprecationWarning: the rpc synchronous context manager is deprecated
  with self.rpc(self.nanny) as r:
/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: Deprecati

    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/asyncio/tasks.py", line 445, in wait_for
    return fut.result()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/comm/tcp.py", line 241, in read
    convert_stream_closed_error(self, e)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc.__class__.__name__}: {exc}") from exc
distributed.comm.core.CommClosedError: in <TCP (closed)  local=tcp://127.0.0.1:54928 remote=tcp://127.0.0.1:54825>: ConnectionResetError: [Errno 54] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-

2022-09-15 15:22:17,389 - distributed.nanny - WARNING - Worker process still alive after 3.1999992370605472 seconds, killing
2022-09-15 15:22:17,399 - distributed.nanny - WARNING - Worker process still alive after 3.1999992370605472 seconds, killing
2022-09-15 15:22:17,410 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing
2022-09-15 15:22:17,412 - distributed.nanny - WARNING - Worker process still alive after 3.2 seconds, killing
2022-09-15 15:22:17,415 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing
2022-09-15 15:22:17,422 - distributed.nanny - WARNING - Worker process still alive after 3.1999902343750004 seconds, killing
2022-09-15 15:22:17,434 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing
2022-09-15 15:22:20,091 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:54811
Traceback (most recent ca

In [12]:
# Check on the client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 9
Total threads: 18,Total memory: 57.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54791,Workers: 9
Dashboard: http://127.0.0.1:8787/status,Total threads: 18
Started: 2 minutes ago,Total memory: 57.60 GiB
Comm: tcp://127.0.0.1:54821,Total threads: 2
Dashboard: http://127.0.0.1:54822/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:54797,


/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/worker.py:1490: DeprecationWarning: the rpc synchronous context manager is deprecated
  with self.rpc(self.nanny) as r:
2022-09-15 15:22:33,264 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing
2022-09-15 15:22:44,918 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:54879
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/comm/core.py", line 291, in connect
    comm = await asyncio.wait_for(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sail_act_pyart/lib/python3.10/asyncio/tasks.py", line 445, in wait_for
    return fut.result()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sai